In [1]:
import numpy as np
from clean_scripts import *
import scipy
import sys
import pickle
import pandas as pd

sys.path.append('clean_scripts')
import player_shotInfo2

/Users/geoffreypoon/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
bins, binRange = ([25,18], [[-250,250], [-47.5,312.5]])

hist2d, xedges, yedges, binnumber = scipy.stats.binned_statistic_2d([0], [0], 
                                                                    [0],
                                                                    statistic='sum',
                                                                    bins=bins, 
                                                                    range=binRange)

binDat = (bins, binRange, xedges, yedges, binnumber)
X_T, Y_T = np.meshgrid(xedges[:-1] + 10, yedges[:-1] + 10)
X, Y = (X_T.T, Y_T.T)
# X, Y = np.meshgrid(xedges[:-1], yedges[:-1])

In [3]:
X.shape

(25, 18)

In [4]:
print(xedges[-1]-xedges[0], yedges[-1]-yedges[0])

500.0 360.0


In [5]:
# W_norm_att = np.loadtxt('clean_scripts/W_norm_SHOT_ATTEMPTED_FLAG.txt')
# H_norm_att = np.loadtxt('clean_scripts/H_norm_SHOT_ATTEMPTED_FLAG.txt')

# W_norm_made = np.loadtxt('clean_scripts/W_norm_SHOT_MADE_FLAG.txt')
# H_norm_made = np.loadtxt('clean_scripts/H_norm_SHOT_MADE_FLAG.txt')

# with open ('clean_scripts/top_players_nameList.pickle', 'rb') as fp:
#     top_players_nameList = pickle.load(fp)

In [6]:
import os
import simplejson as json
from clean_scripts import nmf_func as nmf


season = '2016-17'
phi = 30

######################

shots_path = os.path.abspath(os.getcwd()) + '/data/shots/shots_data_%s.json'%season
with open(shots_path, 'r') as json_data:
    d = json.load(json_data)
    json_data.close()

shots_dat = pd.DataFrame(d['resultSets'][0]['rowSet'], 
                         columns=d['resultSets'][0]['headers'])

df = pd.DataFrame(shots_dat, 
                  columns = ['PLAYER_ID', 'PLAYER_NAME', 
                             'TEAM_ID', 'TEAM_NAME', 'ACTION_TYPE',
                             'SHOT_DISTANCE', 'SHOT_TYPE', 
                             'LOC_X', 'LOC_Y', 
                             'SHOT_ATTEMPTED_FLAG', 'SHOT_MADE_FLAG'])

num_players = 300
top_players_shotNum = df.PLAYER_NAME.value_counts()[:num_players]
top_players_nameList = top_players_shotNum.index.tolist()




W_norm, H_norm = (dict(), dict())
for flag_name in ['SHOT_ATTEMPTED_FLAG', 'SHOT_MADE_FLAG']:
    directory = 'clean_scripts/%s/%s/shotHist_LGCP_phi%d_seed546682'%(season, flag_name, phi)
    LL = np.zeros((len(top_players_nameList), np.prod(bins)))
    for i, player in enumerate(top_players_nameList):
        lambdaN_v = np.loadtxt(directory + '/lambda_%s.txt'%(player))
        norm_lambdaN_v = lambdaN_v / np.sum(lambdaN_v)
        LL[i,:] = norm_lambdaN_v[:]

    n_features = 10
    W_norm[flag_name], H_norm[flag_name] = nmf.run(LL, n_features=n_features)

    
######################


In [7]:
    
player = 'Stephen Curry'
player_index = top_players_nameList.index(player)
print(player_index, top_players_nameList[player_index])


directory = 'clean_scripts/%s/%s/shotHist_LGCP_phi30_seed546682/'
shot_made_src = (directory + 'lambda_%s.txt') %(season, 'SHOT_MADE_FLAG', player)
shot_att_src = (directory + 'lambda_%s.txt') %(season, 'SHOT_ATTEMPTED_FLAG', player)

shots_made_map = np.loadtxt(shot_made_src)
shots_att_map = np.loadtxt(shot_att_src)

tot_shots_made = shots_made_map.sum()
tot_shots_att = shots_att_map.sum()

print('Predicted FG%: ', tot_shots_made / tot_shots_att)
    
######################

nmf_made_map = np.dot(W_norm['SHOT_MADE_FLAG'], H_norm['SHOT_MADE_FLAG'])[player_index, :]
nmf_att_map = np.dot(W_norm['SHOT_ATTEMPTED_FLAG'], H_norm['SHOT_ATTEMPTED_FLAG'])[player_index, :]

nmf_FGper_map = (tot_shots_made / tot_shots_att) * (nmf_made_map / nmf_att_map)

9 Stephen Curry
Predicted FG%:  0.4797179150904743


In [8]:
def create_dat_arr(X, Y, fg_vec, att_vec, bottom_frac=0.02):
    def calc_threshold_att_prob(att_vec, bottom_frac):
        sorted_att_prob = np.array(sorted(att_vec, reverse=True))
        sorted_att_prob_cumsum = np.cumsum(sorted_att_prob)
        filtered_att_prob = sorted_att_prob[1-sorted_att_prob_cumsum >= bottom_frac]
#         print(filtered_att_prob[-1])
        return filtered_att_prob[-1]
    
    def mask_vec(vec, att_vec, bottom_frac):
        thresh = calc_threshold_att_prob(att_vec, bottom_frac)
        masked_vec = np.copy(vec)
        masked_vec[att_vec < thresh] = float('nan')
        return masked_vec
    
    masked_att_vec = mask_vec(att_vec, att_vec, bottom_frac)
    masked_fg_vec = mask_vec(fg_vec, att_vec, bottom_frac)
    arr = np.hstack((
        X.reshape(-1,1), Y.reshape(-1,1), 
        masked_att_vec.reshape(-1,1), masked_fg_vec.reshape(-1,1)
    ))
    arr[:,2:] = arr[:,2:]*100
    return arr

arr = create_dat_arr(X, Y, nmf_FGper_map, nmf_att_map, bottom_frac=0.05)

In [9]:
df = pd.DataFrame(arr, columns=['x', 'y', 'fga_perc', 'fgm_perc'])
df.head()

,x,y,fga_perc,fgm_perc
0,-240.0,-37.5,NaN,NaN
1,-240.0,-17.5,0.435436,35.869245
2,-240.0,2.5,1.174981,40.528166
3,-240.0,22.5,0.678524,51.608756
4,-240.0,42.5,0.394344,46.194559


In [10]:
np.sum(df.fga_perc)

94.96038357089935

In [11]:
arr = create_dat_arr(X, Y, nmf_FGper_map, nmf_att_map, bottom_frac=0.05)
df = pd.DataFrame(arr, columns=['x', 'y', 'fga_perc', 'fgm_perc'])
df_noNAN = df[np.isfinite(df.fga_perc)]

In [12]:
column = 'fga_perc'
df_temp = df_noNAN.copy()
df_temp['plot'] = df_noNAN[column] * (100./df_noNAN[column].max())
df_temp.describe()

,x,y,fga_perc,fgm_perc,plot
count,259.000000,259.000000,259.000000,259.000000,259.000000
mean,1.235521,129.835907,0.366642,43.959457,4.498810
std,135.228842,82.950947,0.690719,10.566299,8.475327
min,-240.000000,-17.500000,0.071870,14.777549,0.881868
25%,-120.000000,62.500000,0.101789,36.971462,1.248985
50%,0.000000,122.500000,0.150478,42.640457,1.846404
75%,120.000000,202.500000,0.357675,49.826123,4.388780
max,240.000000,282.500000,8.149765,89.800934,100.000000


In [13]:
np.nanmax([1, np.nan, 1])
pd.Series([1, np.nan, 1, 3]).max()

3.0

In [14]:
import numpy as np

from bokeh.plotting import figure, output_notebook, show
from bokeh.layouts import layout, widgetbox
from bokeh.models import ColumnDataSource, HoverTool, Div, CustomJS
from bokeh.models import ColorBar, LinearColorMapper, Plot, Range1d, LinearAxis, FixedTicker, FuncTickFormatter

from bokeh.models.widgets import Slider, Select, TextInput
from bokeh.io import curdoc
import bokeh.palettes

output_notebook()

alpha = 0.7

# Create color map
colors = bokeh.palettes.magma(256)[::-1]
mapper = LinearColorMapper(palette = colors, nan_color = 'white',
                           low=0., high=100.)
ticker = FixedTicker(ticks=[5, 95])
formatter = FuncTickFormatter(code="""
    data = {5: 'low', 95: 'high'}
    return data[tick]
""")
cbar = ColorBar(color_mapper=mapper, ticker=ticker, formatter=formatter,
                major_label_text_align='left',
                major_label_text_font_size='10pt', label_standoff=6,
                border_line_color=None, location=(0, 0),
                orientation="horizontal",
                scale_alpha=alpha
)




# Create Column Data Source that will be used by the plot
column = 'fga_perc'
df_src = df.copy()
df_src['plot'] = df_src[column] * (100./df_src[column].max())
source = ColumnDataSource(data=df_src)

# hover = HoverTool(tooltips=[
#     ('(X, Y)', '(@X, @Y)'),
#     ('FGA% (rel. avg.)', '@FGA_perc%'),
#     ('FG% (rel. avg.)', '@FGM_perc%'),
# ])

hover = HoverTool(
    names = ["heatmap"],
    tooltips=[
                ('(X, Y)', '(@x, @y)'),
                ('FGA% (rel. avg.)', '@fga_perc%'),
                ('FG% (rel. avg.)', '@fgm_perc%')
    ]
)

p3 = figure(
        x_range=(-250,250), y_range=(-47.5,470-47.5),
        plot_width=500, plot_height=550,
        tools=[hover], toolbar_location='below'
)
p3.rect(
    x = "x", y = "y", width=20, height=20,
    source = source,
    fill_color = {'field': 'plot', 'transform': mapper},
    fill_alpha = alpha,
    line_color = None,
    name='heatmap'
)
p3.add_layout(cbar, 'above')
    
# callback_season = CustomJS(args=dict(source=source), code="""
#     var season = cb_obj.value
#     console.log(season);
#     source.change.emit();
# """)

def callback_season(source=source, window=None):
    data = source.data
    season = int(cb_obj.value)
    x, y = data['x'], data['y']
    print(season)
    source.change.emit()
    
def callback_player(source=source, window=None):
    data = source.data
    player = str(cb_obj.value.strip())
    x, y = data['x'], data['y']
    directory = 'clean_scripts/SHOT_MADE_FLAG/shotHist_LGCP_phi30_seed546682/'
    shot_made_src = directory + 'lambda_' + str(player) + '.txt'
    print(shot_made_src)
    source.change.emit()
    
def callback_col(source=source, window=None):
    data = source.data
    x, y = data['x'], data['y']
    plot = str(cb_obj.value.strip())
    source.change.emit()

    
# Create Input controls
season = Slider(title = "Year season started", 
                start = 2004, end = 2017, value = 2016, step = 1,
                callback = CustomJS.from_py_func(callback_season)
               )
player = TextInput(title = "Player name", 
                   value = 'Stephen Curry',
                   callback = CustomJS.from_py_func(callback_player)
                  )
plot_col = TextInput(title="Color by", 
                     value='fga_perc',
                     callback = CustomJS.from_py_func(callback_col)
                    )


controls = [season, player, plot_col]

#for control in controls:
 #   control.on_change('value', lambda attr, old, new: update())
sizing_mode = 'fixed'  # 'fixed' or scale_width' looks nice with this example

inputs = widgetbox(*controls, sizing_mode=sizing_mode)
l = layout([
    [inputs], [p3]
], sizing_mode=sizing_mode)

# curdoc().title = "ShotChart"
show(l)

Loading BokehJS ...

In [70]:
import pandas as pd
import numpy as np
import os
import scipy
import re

#################

bins, binRange = ([25,18], [[-250,250], [-47.5,312.5]])

hist2d, xedges, yedges, binnumber = scipy.stats.binned_statistic_2d([0], [0], 
                                                                    [0],
                                                                    statistic='sum',
                                                                    bins=bins, 
                                                                    range=binRange)

binDat = (bins, binRange, xedges, yedges, binnumber)
X_T, Y_T = np.meshgrid(xedges[:-1] + 10, yedges[:-1] + 10)
X, Y = (X_T.T, Y_T.T)

#################

unique_players = set()

for yr in range(2010,2018):
    season = '%d-%d'%(yr, yr-2000+1)
    for flag in ['SHOT_ATTEMPTED_FLAG']:
        directory = 'clean_scripts/%s/%s/shotHist_LGCP_phi30_seed546682/' %(season, flag)
        for file in os.listdir(directory):
            if file.startswith("lambda") and file.endswith(".txt"):
                name = re.split('_|\.', file)[1]
                unique_players.add(name)

###################

list_of_playerSeason = []
LL = {'SHOT_ATTEMPTED_FLAG':[], 'SHOT_MADE_FLAG':[]}
for yr in range(2010,2018):
    season = '%d-%d'%(yr, yr-2000+1)
    for flag in ['SHOT_ATTEMPTED_FLAG', 'SHOT_MADE_FLAG']:
        directory = 'clean_scripts/%s/%s/shotHist_LGCP_phi30_seed546682/' %(season, flag)
        for file in os.listdir(directory):
            if file.startswith("lambda") and file.endswith(".txt"):
                name = re.split('_|\.', file)[1]
                if flag == 'SHOT_ATTEMPTED_FLAG':
                    list_of_playerSeason.extend(['%s_%s' %(season, name)])
                LL[flag].append(np.loadtxt(directory + file))
for flag in ['SHOT_ATTEMPTED_FLAG', 'SHOT_MADE_FLAG']:
    LL[flag] = np.array(LL[flag])

In [16]:
# print(LL['SHOT_ATTEMPTED_FLAG'].shape, LL['SHOT_MADE_FLAG'].shape)

In [17]:
# LL_norm = {'SHOT_ATTEMPTED_FLAG': LL['SHOT_ATTEMPTED_FLAG'].copy(), 
#            'SHOT_MADE_FLAG': LL['SHOT_MADE_FLAG'].copy()}
# for flag in ['SHOT_ATTEMPTED_FLAG', 'SHOT_MADE_FLAG']:
#     for i in range(len(LL_norm[flag])):
#         LL_norm[flag][i,:] = LL_norm[flag][i,:] / LL_norm[flag][i,:].sum()

In [18]:
# from clean_scripts import nmf_func as nmf

# n_features = 10
# W_att, H_att = nmf.run(LL['SHOT_ATTEMPTED_FLAG'], n_features=n_features)
# W_made, H_made = nmf.run(LL['SHOT_MADE_FLAG'], n_features=n_features)

In [19]:
# H_att_norm = H_att.copy()
# W_att_norm = W_att.copy()
# for i in range(n_features):
#     temp = H_att_norm[i,:].sum()
#     H_att_norm[i,:] = H_att_norm[i,:] / temp
#     W_att_norm[:,i] = W_att_norm[:,i] * temp
    
# H_made_norm = H_made.copy()
# W_made_norm = W_made.copy()
# for i in range(n_features):
#     temp = H_made_norm[i,:].sum()
#     H_made_norm[i,:] = H_made_norm[i,:] / temp
#     W_made_norm[:,i] = W_made_norm[:,i] * temp

In [20]:
# from clean_scripts import nmf_func as nmf

# n_features16 = 16
# W_att16, H_att16 = nmf.run(LL['SHOT_ATTEMPTED_FLAG'], n_features=n_features16)
# W_made16, H_made16 = nmf.run(LL['SHOT_MADE_FLAG'], n_features=n_features16)

In [21]:
# H_att16_norm = H_att16.copy()
# W_att16_norm = W_att16.copy()
# for i in range(n_features16):
#     temp = H_att16_norm[i,:].sum()
#     H_att16_norm[i,:] = H_att16_norm[i,:] / temp
#     W_att16_norm[:,i] = W_att16_norm[:,i] * temp
    
# H_made16_norm = H_made16.copy()
# W_made16_norm = W_made16.copy()
# for i in range(n_features16):
#     temp = H_made16_norm[i,:].sum()
#     H_made16_norm[i,:] = H_made16_norm[i,:] / temp
#     W_made16_norm[:,i] = W_made16_norm[:,i] * temp

In [22]:
# from clean_scripts import nmf_func as nmf

# n_features20 = 20
# W_att20, H_att20 = nmf.run(LL['SHOT_ATTEMPTED_FLAG'], n_features=n_features20)
# W_made20, H_made20 = nmf.run(LL['SHOT_MADE_FLAG'], n_features=n_features20)

In [23]:
# H_att20_norm = H_att20.copy()
# W_att20_norm = W_att20.copy()
# for i in range(n_features20):
#     temp = H_att20_norm[i,:].sum()
#     H_att20_norm[i,:] = H_att20_norm[i,:] / temp
#     W_att20_norm[:,i] = W_att20_norm[:,i] * temp
    
# H_made20_norm = H_made20.copy()
# W_made20_norm = W_made20.copy()
# for i in range(n_features20):
#     temp = H_made20_norm[i,:].sum()
#     H_made20_norm[i,:] = H_made20_norm[i,:] / temp
#     W_made20_norm[:,i] = W_made20_norm[:,i] * temp

In [72]:
import pickle
import copy

# nmf_dict = {}
# nmf_dict['LL'] = copy.deepcopy(LL)
# nmf_dict['LL_norm'] = copy.deepcopy(LL_norm)
# #-----------------
# nmf_dict['H_att10_norm'] = H_att_norm.copy()
# nmf_dict['W_att10_norm'] = W_att_norm.copy()
# nmf_dict['H_made10_norm'] = H_made_norm.copy()
# nmf_dict['W_made10_norm'] = W_made_norm.copy()
# #-----------------
# nmf_dict['H_att16_norm'] = H_att16_norm.copy()
# nmf_dict['W_att16_norm'] = W_att16_norm.copy()
# nmf_dict['H_made16_norm'] = H_made16_norm.copy()
# nmf_dict['W_made16_norm'] = W_made16_norm.copy()
# #-----------------
# nmf_dict['H_att20_norm'] = H_att20_norm.copy()
# nmf_dict['W_att20_norm'] = W_att20_norm.copy()
# nmf_dict['H_made20_norm'] = H_made20_norm.copy()
# nmf_dict['W_made20_norm'] = W_made20_norm.copy()
# #-----------------
# nmf_dict['unique_players'] = copy.deepcopy(unique_players)
# nmf_dict['list_of_playerSeason'] = copy.deepcopy(list_of_playerSeason)


def save_obj(obj, name ):
    with open(name, 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

def load_obj(name ):
    with open(name, 'rb') as f:
        return pickle.load(f)
    

save_obj(nmf_dict, 'nmf_dict.pkl')

In [74]:
nmf_dict = load_obj('nmf_dict.pkl')
nmf_dict.keys()

dict_keys(['LL', 'LL_norm', 'H_att10_norm', 'W_att10_norm', 'H_made10_norm', 'W_made10_norm', 'H_att16_norm', 'W_att16_norm', 'H_made16_norm', 'W_made16_norm', 'H_att20_norm', 'W_att20_norm', 'H_made20_norm', 'W_made20_norm', 'unique_players', 'list_of_playerSeason'])

In [26]:
def create_dat_arr(X, Y, fg_vec, att_vec, bottom_frac=0.02):
    def calc_threshold_att_prob(att_vec, bottom_frac):
        sorted_att_prob = np.array(sorted(att_vec, reverse=True))
        sorted_att_prob_cumsum = np.cumsum(sorted_att_prob)
        filtered_att_prob = sorted_att_prob[1-sorted_att_prob_cumsum >= bottom_frac]
        print(filtered_att_prob[-1])
        return filtered_att_prob[-1]
    
    def mask_vec(vec, att_vec, bottom_frac):
        thresh = calc_threshold_att_prob(att_vec, bottom_frac)
        masked_vec = np.copy(vec)
        masked_vec[att_vec < thresh] = np.nan
        return masked_vec
    
    masked_att_vec = mask_vec(att_vec, att_vec, bottom_frac)
    masked_fg_vec = mask_vec(fg_vec, att_vec, bottom_frac)
    arr = np.hstack((
        X.reshape(-1,1), Y.reshape(-1,1), 
        masked_att_vec.reshape(-1,1), masked_fg_vec.reshape(-1,1)
    ))
    arr[:,2:] = arr[:,2:]*100
    return arr

In [116]:
def calc_threshold_att_prob(att_vec, bottom_frac):
    sorted_att_prob = np.array(sorted(att_vec, reverse=True))
    sorted_att_prob_cumsum = np.cumsum(sorted_att_prob)
    filtered_att_prob = sorted_att_prob[1-sorted_att_prob_cumsum >= bottom_frac]
    return filtered_att_prob[-1]

def mask_vec(vec, att_vec, bottom_frac):
    thresh = calc_threshold_att_prob(att_vec, bottom_frac)
    masked_vec = np.copy(vec)
    masked_vec[att_vec < thresh] = np.nan
    return masked_vec

In [118]:
H.shape

(16, 450)

In [127]:
from bokeh.io import show
from bokeh.models import (
    ColumnDataSource,
    HoverTool,
    LinearColorMapper,
    BasicTicker,
    PrintfTickFormatter,
    ColorBar,
)
from bokeh.plotting import figure, Figure
import bokeh.palettes
from bokeh.layouts import column


H = nmf_dict['H_att16_norm'].copy()
for i in range(len(H)):
    H[i,:] = mask_vec(H[i,:], H[i,:], 0.02)[:]
arr_src = np.hstack((
        X.reshape(-1,1), Y.reshape(-1,1), 
        H.T
    ))
df_src = pd.DataFrame(arr_src, columns=['x', 'y']+['basis_%d'%i for i in range(len(H))])
df_src['plot'] = df_src['basis_0'].copy()

# this is the colormap
colors = bokeh.palettes.magma(256)[::-1]
mapper = LinearColorMapper(palette = colors, nan_color = 'white',
                           low = 0., high = 1.)

source = ColumnDataSource(df_src)

TOOLS = "hover,save,reset,wheel_zoom"

p1 = figure(x_range=(-250,250), y_range=(-47.5,470-47.5),
           plot_width=300, plot_height=330,
           tools=TOOLS, toolbar_location='below'
          )

p1.grid.grid_line_color = None
p1.axis.axis_line_color = None
p1.axis.major_tick_line_color = None
p1.axis.minor_tick_line_color = None
p1.axis.major_label_text_font_size = "0pt"
p1.axis.major_label_standoff = 0

# p1.image_url(url=['nba_court.jpg'], 
#             x=-250, y=-47.5,
#             w=500, h=470-47, anchor="bottom_left"
#            )

alpha=0.7

p1.rect(x="x", y="y", width=20, height=20,
       source=source,
       fill_color={'field': 'plot', 'transform': mapper},
       fill_alpha=alpha,
       line_color=None,
       name="heatmap")



#######################################

lw = 2.5
color = 'black'

# Create the basketball hoop
# Diameter of a hoop is 18" so it has a radius of 9", which is a value
# 7.5 in our coordinate system
p1.circle([0], [0], radius=7.5, 
          line_width=lw, line_color=color, fill_color=None)

# Create backboard
p1.rect([0], [-7.5], width=60, height=0.4, 
        line_width=lw, line_color=color, fill_color=None)

# The paint
# Create the outer box 0f the paint, width=16ft, height=19ft
p1.rect([0], [47.5], width=160, height=190, 
        line_width=lw, line_color=color,
        fill_color=None)
# Create the inner box of the paint, widt=12ft, height=19ft
p1.rect([0], [47.5], width=120, height=190, 
        line_width=lw, line_color=color,
        fill_color=None)

# Create free throw top arc
p1.circle([0], [142.5], radius=60, 
          line_width=lw, line_color=color, fill_color=None)

# Restricted Zone, it is an arc with 4ft radius from center of the hoop
p1.arc([0], [0], radius=40, start_angle=0, end_angle=np.pi, 
       line_width=lw, line_color=color)

# Three point line
# Create the side 3pt lines, they are 14ft long before they begin to arc
p1.line([-220, -220], [-47.5, 92.5],
        line_width=lw, line_color=color)
p1.line([220, 220], [-47.5, 92.5], 
        line_width=lw, line_color=color)
# 3pt arc - center of arc will be the hoop, arc is 23'9" away from hoop
# I just played around with the theta values until they lined up with the 
# threes
p1.arc([0], [0], radius=243, start_angle=np.pi*24.5/180, end_angle=np.pi*155.5/180, 
       line_width=lw, line_color=color)

# Center Court
p1.arc([0], [422.5], radius=60, start_angle=np.pi, end_angle=0,
       line_width=lw, line_color=color)
p1.arc([0], [422.5], radius=20, start_angle=np.pi, end_angle=0,
       line_width=lw, line_color=color)

#######################################


p1.select_one(HoverTool).names = ["heatmap"]
p1.select_one(HoverTool).tooltips = [
    ('(x, y)', '(@x, @y)'),
    ('value', '@plot'),
]


########################################

callback = CustomJS(args=dict(source=source), code="""
    var data = source.data;
    var f = cb_obj.value
    var max_val = 0
    for (i=0;i<data['x'].length;i++){
        data['plot'][i] =  data['basis_' + f.toString()][i]
    }
    for (i=0;i<data['x'].length;i++){
        if(data['plot'][i] > max_val){
            max_val = data['plot'][i];
        }
    }
    for (i=0;i<data['x'].length;i++){
        data['plot'][i] = data['plot'][i] / max_val
    }
    //console.log("basis_" + f.toString())
    source.change.emit();
""")

slider = Slider(start=0, end=len(H)-1, value=0, step=1, title="Attempted shots, Basis vector")
slider.js_on_change('value', callback)

# slider = Slider(start=0, end=len(H)-1, value=0, step=1, title="Basis vector")
# def update(attr=None, old=None, new=None):
#     print('sup')
#     source.data = dict(
#         x = df_src['x'],
#         y = df_src['Y'],
#         plot = df_src['basis_%d'%slider.value],
#     )
# slider.on_change('value', lambda attr, old, new: update())


layout = column(slider, p1)

# show(layout)

show(layout)


In [63]:
df_src.head()

,x,y,basis_0,basis_1,basis_2,basis_3,basis_4,basis_5,basis_6,basis_7,basis_8,basis_9,basis_10,basis_11,basis_12,basis_13,basis_14,basis_15,plot
0,-240.0,-37.5,0.000000e+00,0.000000e+00,0.000000e+00,4.202008e-06,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.003495,0.000000e+00,0.000000e+00,0.0,0.000280,0.000000e+00,0.000461,4.311921e-04,0.000000e+00
1,-240.0,-17.5,1.740092e-08,0.000000e+00,0.000000e+00,0.000000e+00,1.283607e-08,2.149290e-07,0.000000e+00,1.332983e-13,0.049550,6.307468e-07,1.576043e-08,0.0,0.000002,0.000000e+00,0.011032,8.437414e-07,1.740092e-08
2,-240.0,2.5,0.000000e+00,2.269323e-15,0.000000e+00,7.652330e-15,1.292387e-03,0.000000e+00,0.000000e+00,1.841848e-07,0.120657,2.318197e-08,0.000000e+00,0.0,0.000000,0.000000e+00,0.030322,0.000000e+00,0.000000e+00
3,-240.0,22.5,6.133114e-08,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,5.696295e-08,5.826197e-08,0.058626,0.000000e+00,0.000000e+00,0.0,0.000000,8.519798e-17,0.035795,1.447474e-03,6.133114e-08
4,-240.0,42.5,7.759790e-08,0.000000e+00,1.074653e-07,0.000000e+00,0.000000e+00,9.730014e-08,0.000000e+00,0.000000e+00,0.019987,0.000000e+00,0.000000e+00,0.0,0.000000,1.655541e-05,0.026232,2.485010e-03,7.759790e-08


In [57]:
(df_src.basis_4/df_src.basis_4.max()).describe()

count    450.000000
mean       0.061066
std        0.156213
min        0.000000
25%        0.000000
50%        0.000000
75%        0.014228
max        1.000000
Name: basis_4, dtype: float64

In [126]:
from bokeh.io import show
from bokeh.models import (
    ColumnDataSource,
    HoverTool,
    LinearColorMapper,
    BasicTicker,
    PrintfTickFormatter,
    ColorBar,
)
from bokeh.plotting import figure, Figure
import bokeh.palettes
from bokeh.layouts import column


H = nmf_dict['H_made16_norm'].copy()
for i in range(len(H)):
    H[i,:] = mask_vec(H[i,:], H[i,:], 0.02)[:]
arr_src = np.hstack((
        X.reshape(-1,1), Y.reshape(-1,1), 
        H.T
    ))
df_src = pd.DataFrame(arr_src, columns=['x', 'y']+['basis_%d'%i for i in range(len(H))])
df_src['plot'] = df_src['basis_0'].copy()

# this is the colormap
colors = bokeh.palettes.magma(256)[::-1]
mapper = LinearColorMapper(palette = colors, nan_color = 'white',
                           low = 0., high = 1.)

source = ColumnDataSource(df_src)

TOOLS = "hover,save,reset,wheel_zoom"

p2 = figure(x_range=(-250,250), y_range=(-47.5,470-47.5),
           plot_width=300, plot_height=330,
           tools=TOOLS, toolbar_location='below'
          )

p2.grid.grid_line_color = None
p2.axis.axis_line_color = None
p2.axis.major_tick_line_color = None
p2.axis.minor_tick_line_color = None
p2.axis.major_label_text_font_size = "0pt"
p2.axis.major_label_standoff = 0

# p1.image_url(url=['nba_court.jpg'], 
#             x=-250, y=-47.5,
#             w=500, h=470-47, anchor="bottom_left"
#            )

alpha=0.7

p2.rect(x="x", y="y", width=20, height=20,
       source=source,
       fill_color={'field': 'plot', 'transform': mapper},
       fill_alpha=alpha,
       line_color=None,
       name="heatmap")



#######################################

lw = 2.5
color = 'black'

# Create the basketball hoop
# Diameter of a hoop is 18" so it has a radius of 9", which is a value
# 7.5 in our coordinate system
p2.circle([0], [0], radius=7.5, 
          line_width=lw, line_color=color, fill_color=None)

# Create backboard
p2.rect([0], [-7.5], width=60, height=0.4, 
        line_width=lw, line_color=color, fill_color=None)

# The paint
# Create the outer box 0f the paint, width=16ft, height=19ft
p2.rect([0], [47.5], width=160, height=190, 
        line_width=lw, line_color=color,
        fill_color=None)
# Create the inner box of the paint, widt=12ft, height=19ft
p2.rect([0], [47.5], width=120, height=190, 
        line_width=lw, line_color=color,
        fill_color=None)

# Create free throw top arc
p2.circle([0], [142.5], radius=60, 
          line_width=lw, line_color=color, fill_color=None)

# Restricted Zone, it is an arc with 4ft radius from center of the hoop
p2.arc([0], [0], radius=40, start_angle=0, end_angle=np.pi, 
       line_width=lw, line_color=color)

# Three point line
# Create the side 3pt lines, they are 14ft long before they begin to arc
p2.line([-220, -220], [-47.5, 92.5],
        line_width=lw, line_color=color)
p2.line([220, 220], [-47.5, 92.5], 
        line_width=lw, line_color=color)
# 3pt arc - center of arc will be the hoop, arc is 23'9" away from hoop
# I just played around with the theta values until they lined up with the 
# threes
p2.arc([0], [0], radius=243, start_angle=np.pi*24.5/180, end_angle=np.pi*155.5/180, 
       line_width=lw, line_color=color)

# Center Court
p2.arc([0], [422.5], radius=60, start_angle=np.pi, end_angle=0,
       line_width=lw, line_color=color)
p2.arc([0], [422.5], radius=20, start_angle=np.pi, end_angle=0,
       line_width=lw, line_color=color)

#######################################


p2.select_one(HoverTool).names = ["heatmap"]
p2.select_one(HoverTool).tooltips = [
    ('(x, y)', '(@x, @y)'),
    ('value', '@plot'),
]


########################################

callback = CustomJS(args=dict(source=source), code="""
    var data = source.data;
    var f = cb_obj.value
    var max_val = 0
    for (i=0;i<data['x'].length;i++){
        data['plot'][i] =  data['basis_' + f.toString()][i]
    }
    for (i=0;i<data['x'].length;i++){
        if(data['plot'][i] > max_val){
            max_val = data['plot'][i];
        }
    }
    for (i=0;i<data['x'].length;i++){
        data['plot'][i] = data['plot'][i] / max_val
    }
    //console.log("basis_" + f.toString())
    source.change.emit();
""")

slider = Slider(start=0, end=len(H)-1, value=0, step=1, title="Made shots, Basis vector")
slider.js_on_change('value', callback)

# slider = Slider(start=0, end=len(H)-1, value=0, step=1, title="Basis vector")
# def update(attr=None, old=None, new=None):
#     print('sup')
#     source.data = dict(
#         x = df_src['x'],
#         y = df_src['Y'],
#         plot = df_src['basis_%d'%slider.value],
#     )
# slider.on_change('value', lambda attr, old, new: update())


layout = column(slider, p2)

# show(layout)

show(layout)


# Try to plot for players

In [157]:
tot_att = np.sum(nmf_dict['LL']['SHOT_ATTEMPTED_FLAG'], axis=1)
print(tot_att.max(), len(tot_att))
tot_att

1551.0001521371532 2400


array([600.99977545, 332.00047217, 676.99947133, ..., 165.99940483,
       278.99975735,  95.00003973])

In [158]:
tot_made = np.sum(nmf_dict['LL']['SHOT_MADE_FLAG'], axis=1)
print(tot_made.max(), len(tot_made))
tot_made

685.0002088144661 2400


array([255.99996501, 167.9985529 , 306.99836227, ...,  97.00003525,
       129.00051499,  39.00005916])

In [159]:
H_att = nmf_dict['H_att16_norm'].copy()
W_att = nmf_dict['W_att16_norm'].copy()
LL_nmf_att = np.dot(W_att, H_att)
LL_nmf_att.shape

(2400, 450)

In [160]:
H_made = nmf_dict['H_made16_norm'].copy()
W_made = nmf_dict['W_made16_norm'].copy()
LL_nmf_made = np.dot(W_made, H_made)
LL_nmf_made.shape

(2400, 450)

In [164]:
# LGCP+NMF predicted total number of shots 
# is in good agreement with LGCP prediction
np.sum(LL_nmf_att, axis=1)

array([600.9997996 , 332.0007094 , 677.00054384, ..., 165.99901224,
       278.99991908,  94.99995048])

In [167]:
# Gives a good idea of the total number of shots taken in each area
np.sum(LL_nmf_att, axis=0)

array([2.50742261e+02, 3.34123832e+03, 8.79200756e+03, 5.36674226e+03,
       2.76243619e+03, 1.47445473e+03, 8.38007241e+02, 6.52742658e+02,
       3.20820723e+02, 8.34538521e+01, 2.41348322e+01, 9.37466700e+00,
       1.79081316e+00, 3.37213308e+00, 1.53787417e+00, 6.53942042e-01,
       7.12282966e-01, 1.65549074e+00, 1.67211230e+02, 2.04721354e+03,
       6.49348469e+03, 4.22301822e+03, 2.06170720e+03, 1.10647553e+03,
       8.54667631e+02, 1.92174804e+03, 3.21855780e+03, 1.89301238e+03,
       4.98504012e+02, 1.34302929e+02, 3.34254114e+01, 1.44830157e+01,
       6.55019560e+00, 5.85916381e+00, 1.51911350e+00, 3.85329895e+00,
       6.56978856e+01, 6.38745233e+02, 1.83805530e+03, 1.26054931e+03,
       8.00973542e+02, 6.18214024e+02, 5.11268366e+02, 4.63891649e+02,
       1.17395433e+03, 4.16024335e+03, 4.46758075e+03, 1.47570724e+03,
       3.68889924e+02, 1.13601798e+02, 3.22088732e+01, 1.12168720e+01,
       1.32111065e+01, 1.15982489e+01, 1.03998119e+02, 8.32382437e+02,
      

In [151]:
np.sum(W_made, axis=0)

array([91225.24005511, 46076.95455867, 46162.99778987, 32519.8966511 ,
       26574.53209433, 26702.18599513, 26218.27944793, 24804.07881216,
       22797.72248724, 26823.5086862 , 21614.75352703, 27328.20283728,
       20646.05041719, 23096.57362256, 18963.40657246, 21022.59641101])

In [148]:
# H_made.shape
np.sum(H_made, axis=1)

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])

In [143]:
np.sum(LL_nmf_made, axis=1)

array([255.99975346, 167.99873944, 306.99856226, ...,  97.0000992 ,
       129.00061949,  39.0000833 ])

In [155]:
FGperc_nmf = LL_nmf_made/LL_nmf_att
scipy.stats.describe(FGperc_nmf[1,:])

DescribeResult(nobs=450, minmax=(0.002170267444517711, 10.717115949979505), mean=0.6225745764280713, variance=1.175201307556978, skewness=4.621365498049476, kurtosis=27.05387451185733)

In [137]:
def calc_threshold_att_prob(att_vec, bottom_frac):
    sorted_att_prob = np.array(sorted(att_vec, reverse=True))
    sorted_att_prob_cumsum = np.cumsum(sorted_att_prob)
    filtered_att_prob = sorted_att_prob[1.-sorted_att_prob_cumsum >= bottom_frac]
    return filtered_att_prob[-1]

def mask_vec(vec, att_vec, bottom_frac):
    thresh = calc_threshold_att_prob(att_vec, bottom_frac)
    masked_vec = np.copy(vec)
    masked_vec[att_vec < thresh] = np.nan
    return masked_vec

In [142]:
att_vec1 = LL_nmf_att[1,:]
sorted_att_prob1 = np.array(sorted(att_vec1, reverse=True))
sorted_att_prob1

array([6.07552127e+01, 4.99761933e+01, 2.96327774e+01, 2.40818855e+01,
       1.82232246e+01, 1.06161956e+01, 7.24873837e+00, 6.13997315e+00,
       5.33183385e+00, 4.29827377e+00, 4.04905183e+00, 2.93855031e+00,
       2.77258171e+00, 2.55206288e+00, 2.52173832e+00, 2.44809104e+00,
       2.05482965e+00, 2.00341892e+00, 1.99876527e+00, 1.90491356e+00,
       1.89935226e+00, 1.78080002e+00, 1.67959348e+00, 1.65570135e+00,
       1.61551244e+00, 1.59148041e+00, 1.57993639e+00, 1.48647398e+00,
       1.47821325e+00, 1.40611255e+00, 1.34240406e+00, 1.29686099e+00,
       1.28409033e+00, 1.25908965e+00, 1.23908436e+00, 1.23252623e+00,
       1.12427103e+00, 1.05610070e+00, 1.00147509e+00, 9.50585815e-01,
       9.48528261e-01, 9.46056844e-01, 9.36594225e-01, 8.88826549e-01,
       8.68098160e-01, 8.58804556e-01, 8.47158072e-01, 8.18784645e-01,
       8.10325271e-01, 8.10145422e-01, 7.82213034e-01, 7.72261222e-01,
       7.70559040e-01, 7.54467994e-01, 7.48642519e-01, 7.39640533e-01,
      

In [187]:
# Plots: Want to normalize LL and mask away values that are poorly sampled
# Do the same for FGperc
LL_plot = LL_nmf_att.copy()
for i in range(len(LL_plot)):
    LL_plot[i,:] = LL_plot[i,:]/np.sum(LL_plot[i,:])
FG_plot = FGperc_nmf.copy()


for i in range(len(LL_plot)):
    FG_plot[i,:] = mask_vec(FG_plot[i,:], LL_plot[i,:], 0.05)[:]
    LL_plot[i,:] = mask_vec(LL_plot[i,:], LL_plot[i,:], 0.05)[:]
LL_plot.shape

(2400, 450)

In [78]:
unique_players = nmf_dict['unique_players']
list_of_playerSeason = list_of_playerSeason

In [79]:
list_of_playerSeason[:5]

['2010-11_Andre Iguodala',
 '2010-11_Derrick Favors',
 '2010-11_Gerald Wallace',
 '2010-11_CJ Miles',
 '2010-11_Eric Gordon']

In [198]:
arr_player_src = np.hstack((
        X.reshape(-1,1), Y.reshape(-1,1), 
        LL_plot.T, FG_plot.T
))
print(arr_player_src.shape)
columns = ['x', 'y'] + \
            ['%s_%s'%(i,'att') for i in list_of_playerSeason] + \
            ['%s_%s'%(i,'fgp') for i in list_of_playerSeason]
print(len(columns))
df_player_src = pd.DataFrame(arr_player_src, columns=columns)
df_player_src['plot'] = df_player_src['2016-17_Stephen Curry_att'].copy()
df_player_src['player'] = 'Stephen Curry'
df_player_src['season'] = '2016-17'

(450, 4802)
4802


In [205]:
df_player_src[['x', 'y', '2016-17_Stephen Curry_att', 
               'plot', 'player', 'season']].head()

,x,y,2016-17_Stephen Curry_att,plot,player,season
0,-240.0,-37.5,NaN,NaN,Stephen Curry,2016-17
1,-240.0,-17.5,0.003321,0.003321,Stephen Curry,2016-17
2,-240.0,2.5,0.008777,0.008777,Stephen Curry,2016-17
3,-240.0,22.5,0.006581,0.006581,Stephen Curry,2016-17
4,-240.0,42.5,0.003790,0.003790,Stephen Curry,2016-17


In [202]:
df_player_src.to_pickle('shotChart_player_bokeh_df_2010-18.pkl')

In [203]:
pd.read_pickle('shotChart_player_bokeh_df_2010-18.pkl').head()

,x,y,2010-11_Andre Iguodala_att,2010-11_Derrick Favors_att,2010-11_Gerald Wallace_att,2010-11_CJ Miles_att,2010-11_Eric Gordon_att,2010-11_Hedo Turkoglu_att,2010-11_Darrell Arthur_att,2010-11_Gilbert Arenas_att,...,2017-18_Malik Monk_fgp,2017-18_Tyler Ulis_fgp,2017-18_C_fgp,2017-18_Rodney Hood_fgp,2017-18_Alex Len_fgp,2017-18_Yogi Ferrell_fgp,2017-18_Patrick Beverley_fgp,plot,player,season
0,-240.0,-37.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Stephen Curry,2016-17
1,-240.0,-17.5,0.003191,NaN,0.002908,0.001535,0.001200,NaN,NaN,0.002055,...,NaN,NaN,0.844484,NaN,NaN,0.668665,0.115329,0.003321,Stephen Curry,2016-17
2,-240.0,2.5,0.007701,NaN,0.008752,0.006072,0.003042,0.003997,NaN,0.004649,...,0.433882,0.865466,0.475423,0.614444,NaN,0.503256,0.148149,0.008777,Stephen Curry,2016-17
3,-240.0,22.5,0.004062,NaN,0.006150,0.005157,0.001962,0.004200,NaN,0.002320,...,0.153971,0.294571,0.290290,0.378230,NaN,0.331903,0.310669,0.006581,Stephen Curry,2016-17
4,-240.0,42.5,0.002284,NaN,0.002708,0.003705,0.001826,0.003255,NaN,0.002424,...,0.156551,0.212875,0.278989,0.516525,NaN,0.286150,0.301921,0.003790,Stephen Curry,2016-17


In [ ]:
from bokeh.io import show
from bokeh.models import (
    ColumnDataSource,
    HoverTool,
    LinearColorMapper,
    BasicTicker,
    PrintfTickFormatter,
    ColorBar,
)
from bokeh.plotting import figure, Figure
import bokeh.palettes
from bokeh.layouts import column


df_player_src = pd.read_pickle('shotChart_player_bokeh_df_2010-18.pkl')

# this is the colormap
colors = bokeh.palettes.magma(256)[::-1]
mapper = LinearColorMapper(palette = colors, nan_color = 'white',
                           low = 0., high = 1.)

source = ColumnDataSource(df_player_src)

TOOLS = "hover,save,reset,wheel_zoom"

p2 = figure(x_range=(-250,250), y_range=(-47.5,470-47.5),
           plot_width=300, plot_height=330,
           tools=TOOLS, toolbar_location='below'
          )

p2.grid.grid_line_color = None
p2.axis.axis_line_color = None
p2.axis.major_tick_line_color = None
p2.axis.minor_tick_line_color = None
p2.axis.major_label_text_font_size = "0pt"
p2.axis.major_label_standoff = 0

# p1.image_url(url=['nba_court.jpg'], 
#             x=-250, y=-47.5,
#             w=500, h=470-47, anchor="bottom_left"
#            )

alpha=0.7

p2.rect(x="x", y="y", width=20, height=20,
       source=source,
       fill_color={'field': 'plot', 'transform': mapper},
       fill_alpha=alpha,
       line_color=None,
       name="heatmap")



#######################################

lw = 2.5
color = 'black'

# Create the basketball hoop
# Diameter of a hoop is 18" so it has a radius of 9", which is a value
# 7.5 in our coordinate system
p2.circle([0], [0], radius=7.5, 
          line_width=lw, line_color=color, fill_color=None)

# Create backboard
p2.rect([0], [-7.5], width=60, height=0.4, 
        line_width=lw, line_color=color, fill_color=None)

# The paint
# Create the outer box 0f the paint, width=16ft, height=19ft
p2.rect([0], [47.5], width=160, height=190, 
        line_width=lw, line_color=color,
        fill_color=None)
# Create the inner box of the paint, widt=12ft, height=19ft
p2.rect([0], [47.5], width=120, height=190, 
        line_width=lw, line_color=color,
        fill_color=None)

# Create free throw top arc
p2.circle([0], [142.5], radius=60, 
          line_width=lw, line_color=color, fill_color=None)

# Restricted Zone, it is an arc with 4ft radius from center of the hoop
p2.arc([0], [0], radius=40, start_angle=0, end_angle=np.pi, 
       line_width=lw, line_color=color)

# Three point line
# Create the side 3pt lines, they are 14ft long before they begin to arc
p2.line([-220, -220], [-47.5, 92.5],
        line_width=lw, line_color=color)
p2.line([220, 220], [-47.5, 92.5], 
        line_width=lw, line_color=color)
# 3pt arc - center of arc will be the hoop, arc is 23'9" away from hoop
# I just played around with the theta values until they lined up with the 
# threes
p2.arc([0], [0], radius=243, start_angle=np.pi*24.5/180, end_angle=np.pi*155.5/180, 
       line_width=lw, line_color=color)

# Center Court
p2.arc([0], [422.5], radius=60, start_angle=np.pi, end_angle=0,
       line_width=lw, line_color=color)
p2.arc([0], [422.5], radius=20, start_angle=np.pi, end_angle=0,
       line_width=lw, line_color=color)

#######################################


p2.select_one(HoverTool).names = ["heatmap"]
p2.select_one(HoverTool).tooltips = [
    ('(x, y)', '(@x, @y)'),
    ('value', '@plot'),
]


########################################

callback = CustomJS(args=dict(source=source), code="""
    var data = source.data;
    var f = cb_obj.value
    var max_val = 0
    for (i=0;i<data['x'].length;i++){
        data['plot'][i] =  data['basis_' + f.toString()][i]
    }
    for (i=0;i<data['x'].length;i++){
        if(data['plot'][i] > max_val){
            max_val = data['plot'][i];
        }
    }
    for (i=0;i<data['x'].length;i++){
        data['plot'][i] = data['plot'][i] / max_val
    }
    //console.log("basis_" + f.toString())
    source.change.emit();
""")

slider = Slider(start=0, end=len(H)-1, value=0, step=1, title="Made shots, Basis vector")
slider.js_on_change('value', callback)

# slider = Slider(start=0, end=len(H)-1, value=0, step=1, title="Basis vector")
# def update(attr=None, old=None, new=None):
#     print('sup')
#     source.data = dict(
#         x = df_src['x'],
#         y = df_src['Y'],
#         plot = df_src['basis_%d'%slider.value],
#     )
# slider.on_change('value', lambda attr, old, new: update())


layout = column(slider, p2)

# show(layout)

show(layout)
